In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.models import Model, Sequential
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

import sys
sys.path.append("../")

from pre_processing import creazione_modello_GloVe

In [3]:
dataset = pickle.load(open("dataset/dataset_SLS.pickle", "rb"))

In [4]:
Y = np.array(dataset["sentiment"])

In [5]:
x_train,x_test,y_train,y_test = train_test_split(dataset["sentence"],Y, test_size=0.1, shuffle=True)

In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2700,) (2700,)
(300,) (300,)


In [7]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  1343
numero di negativi nel train:  1357
numero di positivi nel test:  157
numero di negativi nel test:  143


In [8]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [9]:
tokenizer = Tokenizer(num_words=140000)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [10]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=300, weights=[embedding_matrix])

In [12]:
Bi_LSTM_model = Sequential()
Bi_LSTM_model.add(embedding_layer)
Bi_LSTM_model.add(Bidirectional(LSTM(units=32)))
Bi_LSTM_model.add(Dense(32))
Bi_LSTM_model.add(Dense(32))
Bi_LSTM_model.add(Dense(2, activation="softmax"))

#non è necessario modificare l'ottimizzatore di default, poichè già lr=0.001
Bi_LSTM_model.compile(loss="categorical_crossentropy", optimizer=Adagrad(learning_rate=0.05), metrics=["accuracy"])
Bi_LSTM_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 50)           247900    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                21248     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 272,350
Trainable params: 272,350
Non-trainable params: 0
_________________________________________________________________


In [13]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=300)

In [14]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [15]:
Bi_LSTM_model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=24, batch_size=64, verbose=1, validation_data=(x_train_index[2100:], y_train_categorical[2100:]))

Epoch 1/24
33/33 [==============================] - 8s 165ms/step - loss: 0.6710 - accuracy: 0.5819 - val_loss: 0.6603 - val_accuracy: 0.5817
Epoch 2/24
33/33 [==============================] - 5s 146ms/step - loss: 0.6146 - accuracy: 0.6776 - val_loss: 0.5649 - val_accuracy: 0.7067
Epoch 3/24
33/33 [==============================] - 5s 146ms/step - loss: 0.5852 - accuracy: 0.7014 - val_loss: 0.5425 - val_accuracy: 0.7283
Epoch 4/24
33/33 [==============================] - 5s 150ms/step - loss: 0.5277 - accuracy: 0.7429 - val_loss: 0.5580 - val_accuracy: 0.7100
Epoch 5/24
33/33 [==============================] - 5s 142ms/step - loss: 0.5381 - accuracy: 0.7314 - val_loss: 0.5363 - val_accuracy: 0.7400
Epoch 6/24
33/33 [==============================] - 5s 143ms/step - loss: 0.5078 - accuracy: 0.7514 - val_loss: 0.4924 - val_accuracy: 0.7683
Epoch 7/24
33/33 [==============================] - 5s 142ms/step - loss: 0.4746 - accuracy: 0.7767 - val_loss: 0.4726 - val_accuracy: 0.7833
Epoch 

In [16]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=300)

In [17]:
y_pred = Bi_LSTM_model.predict(x_test_index)

In [18]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.72      0.94      0.81       143
           1       0.92      0.67      0.77       157

    accuracy                           0.80       300
   macro avg       0.82      0.80      0.79       300
weighted avg       0.83      0.80      0.79       300



In [ ]:
Bi_LSTM_model.save("models/SLS_BiLSTM")